In [4]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['posts']
posts_collection = db['posts']
posts_collection.drop()




In [5]:
from itertools import islice
import json
from datetime import datetime

with open ('data.jsonl', 'r') as file:
    for line in islice(file, 1000):
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [7]:
#                                                    1.1 Insertar 5 nuevos documentos de usuario con datos únicos.

# Datos únicos para insertar
nuevos_usuarios = [
    {"user": "Cristopher Acuña", "user_email": "usuario1@example.com", "user_age": 25,"post": {"title": "ejemplo1", "content": "ejemplo1 ", "timestamp": "2024-03-11T13:33:57.415698"}, "comments": [{"user": "ejemplo1", "user_email": "usuario1@example.org", "user_age": 39, "comment": {"text": "Texto de ejemplo", "timestamp": "1998-07-21T12:59:07.588567"}}]},
    {"user": "Samuel", "user_email": "usuario2@example.com", "user_age": 30,"post": {"title": "ejemplo2", "content": "ejemplo2 ", "timestamp": "2024-03-11T13:33:57.415698"}},
    {"user": "Emmanuel", "user_email": "usuario3@example.com", "user_age": 35,"post": {"title": "ejemplo3", "content": "ejemplo3 ", "timestamp": "2024-03-11T13:33:57.415698"}},
    {"user": "Antonio", "user_email": "usuario4@example.com", "user_age": 40,"post": {"title": "ejemplo4", "content": "ejemplo4 ", "timestamp": "2024-03-11T13:33:57.415698"}},
    {"user": "Messi", "user_email": "usuario5@example.com", "user_age": 45,"post": {"title": "ejemplo5", "content": "ejemplo5 ", "timestamp": "2024-03-11T13:33:57.415698"}}
]

# Insertar los nuevos usuarios en la colección
result = posts_collection.insert_many(nuevos_usuarios)

print("Documentos insertados:", result.inserted_ids)



Documentos insertados: [ObjectId('65ef8683b6950b889b3ebb79'), ObjectId('65ef8683b6950b889b3ebb7a'), ObjectId('65ef8683b6950b889b3ebb7b'), ObjectId('65ef8683b6950b889b3ebb7c'), ObjectId('65ef8683b6950b889b3ebb7d')]


In [9]:
#                                                  1.5. Eliminar un documento de usuario y todos sus comentarios asociados.


# Encuentra el documento de usuario que deseas eliminar (por ejemplo, por su nombre)
usuario_a_eliminar = posts_collection.find_one({"user": "Messi"})

if usuario_a_eliminar:
    # Elimina el usuario y todos sus comentarios asociados
    posts_collection.delete_one({"_id": usuario_a_eliminar['_id']})
    # Elimina los comentarios asociados al usuario
    posts_collection.delete_many({"post_id": usuario_a_eliminar['_id']})

    print("El usuario y todos sus comentarios asociados han sido eliminados.")
else:
    print("No se encontró el usuario especificado.")



No se encontró el usuario especificado.


In [20]:
# 2.2 Buscar publicaciones creadas después de una fecha específica (por ejemplo, 2000-01-01).

list(db.posts.aggregate([
    {'$addFields': {
        'post_date': {
            '$toDate': '$post.timestamp'
        }
    }},
    {'$project': {
        '_id': 0,
        'user': 1,
        'post.title': 1,
        'post.content': 1,
        'post_date': 1
    }},
    {'$match': {
        'post_date': {'$gte': datetime(2000, 1, 1)}
    }}
]))



[{'user': 'Lindsay Thornton',
  'post': {'title': 'Random title for Lindsay Thornton',
   'content': 'Random content for Lindsay Thornton Pressure employee by above law its not. Way open a such whether cut. Election scientist money role wait serious lot. Sort control man go really quality region.'},
  'post_date': datetime.datetime(2001, 12, 22, 13, 33, 57, 415000)},
 {'user': 'Lindsay Thornton',
  'post': {'title': 'Random title for Lindsay Thornton',
   'content': 'Random content for Lindsay Thornton Matter magazine baby how raise lay goal. Develop catch information establish us beat.\nCommon detail service weight wait. Night people speech nice education how all.'},
  'post_date': datetime.datetime(2012, 10, 8, 1, 16, 1, 836000)},
 {'user': 'Joann Bowen',
  'post': {'title': 'Random title for Joann Bowen',
   'content': 'Random content for Joann Bowen Mind the form local easy evidence. Get leave media good. Contain sort heavy like amount. Point arrive nature modern rock me imagine as

In [23]:

# 3.4 Consulta para buscar usuarios con al menos un comentario
list(
    posts_collection.aggregate([
    {'$project': {
        '_id': 0,
        'user': 1,
        'post.title': 1,
        'post.content': 1,
        'comments': 1
    }},
    {'$match': {
        'comments': {'$exists': True}
    }}
])
)



[{'user': 'Lindsay Thornton',
  'post': {'title': 'Random title for Lindsay Thornton',
   'content': 'Random content for Lindsay Thornton Pressure employee by above law its not. Way open a such whether cut. Election scientist money role wait serious lot. Sort control man go really quality region.'},
  'comments': [{'user': 'Kristi Cooley',
    'user_email': 'claytoncody@example.org',
    'user_age': 39,
    'comment': {'text': 'Random text Capital measure anyone provide PM instead few.\nThan hotel study especially once. Performance eye right president minute admit treat. Before mind would stay role garden.',
     'timestamp': datetime.datetime(1998, 7, 21, 12, 59, 7, 588000)}}]},
 {'user': 'Lindsay Thornton',
  'post': {'title': 'Random title for Lindsay Thornton',
   'content': 'Random content for Lindsay Thornton Career real suggest consider present industry. Else pattern leader. Economy course change yes same. Bill record my rise herself spend door.'},
  'comments': [{'user': 'Beth 

In [25]:
#                                                          4.1 Ordenar todos los usuarios por edad en orden descendente.

usuarios_ordenados_por_edad = posts_collection.aggregate([
    {'$sort': {
        'user_age': -1  # Ordenar por la edad en orden descendente (-1)
    }},
    {'$project': {
        '_id': 0,
        'user': 1,
        'user_age': 1
    }}
])

# Imprimir los resultados
for usuario in usuarios_ordenados_por_edad:
    print(usuario)



{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'user_age': 53}
{'user': 'Richard Bonilla', 'use

In [28]:
#                                                                              5.2. Encontrar al usuario con el contenido de publicación más largo (en función del número de caracteres).
list(
    posts_collection.aggregate([
    {'$project': {
        'user': 1,
        'content_length': {'$strLenCP': '$post.content'}  # Calcula la longitud del contenido de la publicación
    }},
    {'$sort': {
        'content_length': -1  # Ordena por longitud del contenido de la publicación en orden descendente
    }},
    {'$limit': 1}  # Limita los resultados a uno
])
)

[{'_id': ObjectId('65ef80a5b6950b889b3ebb5a'),
  'user': 'Christopher Jenkins',
  'content_length': 238}]

In [29]:
#                                                                              6.1 Incrementar la edad de todos los usuarios en 1.
result = posts_collection.update_many({}, {'$inc': {'user_age': 1}})

print("Se sumó uno a la edad de", result.modified_count, "usuarios.")



Se actualizaron 1009 usuarios.


In [59]:
#                                                                      6.2 Establecer el campo ”contenido” de una publicación específica en una cadena vacía.
import pandas as pd

titulo_publicacion = "ejemplo2"

# Actualizar el campo "contenido" de la publicación específica a una cadena vacía
posts_collection.update_one({"post.title": titulo_publicacion}, {"$set": {"post.content": ""}})

result = posts_collection.find({"post.content": ""})
df = pd.DataFrame(list(result))
df










,_id,user,user_email,user_age,post,comments
0,65ef80a3b6950b889b3eb79b,Mrs. Christine Sawyer,edwardsmeagan@example.com,38,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Christopher Jenkins', 'user_email':..."
1,65ef84f9b6950b889b3ebb74,Usuario1,usuario1@example.com,26,"{'title': 'ejemplo1', 'content': '', 'timestam...","[{'user': 'ejemplo1', 'user_email': 'usuario1@..."
2,65ef84f9b6950b889b3ebb75,Usuario2,usuario2@example.com,31,"{'title': 'ejemplo2', 'content': '', 'timestam...",NaN
